In [140]:
from common import *
from pathlib import Path
import json

In [123]:
logavg = lambda x,y: math.expm1((math.log1p(x)+math.log1p(y))/2)
keepSigFig = lambda n: lambda x: round(x, -int(math.floor(math.log10(abs(x)))) + (n - 1)) if x else x

In [124]:
class PSD(O()):
    class Discrete(O()):
        raw = {
            ('max_depth','num_leaves'): #10
                [(6,1<<6),(9,1<<7),(9,1<<9),(12,1<<8),(12,1<<10),(12,1<<12),(-1,1<<8),(-1,1<<10),(-1,1<<12),(-1,1<<14)],
            #('max','num'): #10
            #    [(6,6),(9,7),(9,9),(12,8),(12,10),(12,12),(-1,8),(-1,10),(-1,12),(-1,14)],
        }
        
    class OneByOne(O()):
        class info(O()):
            a = "main [a]rray data of 3 values, [min mid max]"
            b =  "[b]ack up value, i.e. default value if array doesn't give better results"
            cast = "function to apply to values before using to cast to the right dtype"
            lim = "maximum number of iterations of searching in this hyperparameter"
        class default(O()):
            cast = keepSigFig(2)
            lim = 2
        data = {
            'min_data_in_leaf': O(a=[1,60,375], cast=round),
            'min_sum_hessian_in_leaf': O(a=[0,50,200]),
            'lambda_l1': O(a=[0,.02,.2], b=0, lim=1),
            'lambda_l2': O(a=[0,.02,.2], b=0, lim=1),
        }

class ParamSearch():
    mix = staticmethod(logavg)
    
    def __init__(self, psd):
        self.data = O.mycopy(psd)
        self.setup_data()
        
    def setup_data(self):
        dsc = self.data.Discrete
        dsc.keys = list(flatten( dsc.raw.keys() ))
        dsc.assigns = [list(flatten(x)) for x in product(* dsc.raw.values() )]
        obo = self.data.OneByOne
        for k,v in dict.items(obo.data):
            for i,x in dict.items(obo.default):
                if i not in v:
                    v[i] = x
    
    def search(self):
        dsc, obo = self.data.Discrete, self.data.OneByOne
        for assign in dsc.assigns:
            params = dict(zip(dsc.keys, assign))
            coroutine = self.one_by_one()
            for addon in coroutine:
                params.update(addon)
                coroutine.send((yield params)); assert (yield) == None
        
        
    def one_by_one(self):
        obod = self.data.OneByOne.data
        
        #! main algorithm ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        
        # initialize loop variables
        params = {k: (v.b if 'b' in v else v.a[1]) for k,v in dict.items(obod)}
        ranges = {k: v.a for k,v in dict.items(obod)}
        scores = {k: [-np.inf, -np.inf] for k in params}
        isdone = {k: False for k in params}
        
        # pre loop one-off work
        base_score = yield params; assert (yield) == None
        
        # loop
        for i in range(9999999999):
            #! try new parameter values ############################## part A of loop work
            
            # initialize local loop variable
            new_scores = {k: [-np.inf, -np.inf] for k in params}
            new_params = {k: ( obod[k].cast(self.mix(v[0], v[1])),
                               obod[k].cast(self.mix(v[1], v[2])) ) for k,v in dict.items(ranges)}
            
            # finish condition check
            isdone = {k: v or i>=obod[k].lim for k,v in dict.items(isdone)}
            if all(isdone.values()):
                break
                
            # try new parameter values for all parameters
            for key in list(params):
                if i >= obod[key].lim:
                    continue
                orig = params[key]
                params[key] = new_params[key][0]
                scores[key][0] = yield params; assert (yield) == None
                params[key] = new_params[key][1]
                scores[key][1] = yield params; assert (yield) == None
                params[key] = orig
            
            #! start setting up values for next loop ######################## part B of loop work
            
            # set params to the best found and see if it betters score, updating ranges also
            #num_nochange = 0
            for key in list(params):
                if scores[key][0] > base_score and scores[key][0] >= scores[key][1]:
                    params[key] = new_params[key][0]
                    ranges[key] = [ranges[key][0], params[key], ranges[key][1]]
                elif scores[key][1] > base_score and scores[key][1] >= scores[key][0]:
                    params[key] = new_params[key][1]
                    ranges[key] = [ranges[key][1], params[key], ranges[key][2]]
                else:
                    ranges[key] = [new_params[key][0], ranges[key][1], new_params[key][1]]
                    #num_nochange += 1
                
            # send out new params
            #if num_nochange < len(params):
            base_score = yield params; assert (yield) == None

In [ ]:
class MSD(O()):
    class Features(O()):
        '''features selection groups'''
        data = [
            ('f1','f2','f3'),
            ('f4','f5','f6')
        ]
    
    class Samples(O()):
        '''sample learning/cv split'''
        code = [
            O(method='sklearn', key=44)
        ]
        
    class Params(O()):
        '''parameters constant settings'''
        data = dict(
            objective = 'binary',
            num_iterations = 10000,
            early_stopping_round = 50,
            learning_rate = .05,
            seed = 44,
            bagging_seed = 45,
            feature_fraction_seed = 46,
        )

class ModelSearch():
    def __init__(self, msd):
        # need:
        self.data = msd
    def 

In [148]:
Path('what.txt').stem

'what'

In [ ]:
class ParamManager():
    class io(O()):
        class Params(O()):
            load = lambda p: json.loads(p.read_text())
            save = lambda x: p.write_text(json.dumps(x))
        class Results():
            TODO
        class Extras():
            TODO
            
    class opt(O()):
        class Keep(O()):
            #Params = True # must be True! so therefore not useful to have an option for it
            Results = lambda x: O(score=) TODO
            Extras = False
            
    
    def __init__(self, directory):
        self.dir = directory
        self.opt = O.mycopy(__class__.opt)
        
        # loading the stuff from the file system
        self.paramsList = []
        self.resultsList = [] if self.opt.Keep.Results else None
        self.extrasList = [] if self.opt.Keep.Extras else None
        
        for dot_params in Path(self.dir).glob('*.params'):
            
            i = int(dot_params.stem)
            if i+1 > len(self.paramsList):
                self.paramsList.extend( [None] * (len(self.paramsList)-i-1) )
            
            params = self.io.Params.load(dot_params)
            self.paramsList[i] = params
            
            if self.opt.Keep.Results:
                results = self.io.Results.load(dot_params.parent / (dot_params.stem+'.results'))
                self.resultsList.append(self.opt.Keep.Results(results))
            
            if self.opt.Keep.Extras:
                extras = self.io.Extras.load(dot_params.parent / (dot_params.stem+'.extras'))
                self.extrasList.append(self.opt.Keep.Extras(extras))
                    
            
            
    def get_score(params):
        return self.get_results(params)['score']
        
    def get_results(params):
        TODO
        
    def get_extras(params):
        TODO
        
    def save_run(params, extra):
        TODO
        

class ModelManager():
    def __init__(self, directory):
        self.dir = directory
        self.next_group_index = ?
        self.next_s
    
    def make_param_manager(self, features, samples):
        fset = features if isinstance(features, frozenset) else frozenset(features)
        if fset not in self.fTbl: 
            self.init_features(fset)
        stup = samples; assert isinstance(stup, tuple) and len(stup==3)
        if stup not in self.sTbl:
            self.init_samples(stup)
        fs_ids = (self.fTbl[fset], self.sTbl[stup])
        if fs_ids not in self.fsSet:
            self.init_features_samples_with_ids(*fs_ids)
        return ParamManager()
        
    def init_features(self, fset):
        TODO
    
    def init_samples(self, stup):
        TODO
        
    def init_features_samples_with_ids(fId, sId):
        TODO

In [127]:
s = ParamSearch(PSD)
search = s.search()
cnt = 0
for i,p in zip(range(999),search):
    print(p)
    search.send(i)
    cnt += 1
print(cnt)

{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 10, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 150, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 6.1, 'lambda_l1': 0, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 100.0, 'lambda_l1': 0, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0.01, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0.11, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0, 'lambda_l2': 0.01}
{'max_dept

# SCRATCH TESTING

In [128]:
def hi():
    for i in range(10):
        first = yield i
        assert (yield)==None
        print(' here', first)

In [129]:
h = hi()
for x in h:
    print('x', x)
    h.send(x / 10.)
    #print(next(h))

x 0
 here 0.0
x 1
 here 0.1
x 2
 here 0.2
x 3
 here 0.3
x 4
 here 0.4
x 5
 here 0.5
x 6
 here 0.6
x 7
 here 0.7
x 8
 here 0.8
x 9
 here 0.9
